# Семинар: визуализация данных, EDA (содержит задачи) (25/26 у.год)

## Вступление

На семинаре мы будем заниматься построением различных визуализаций. Наша цель — провести **разведочный анализ данных (exploratory data analysis, EDA)**, чтобы исследовать поведение данных и выявить в них закономерности. Мы продолжим работать с данными о пассажирах Титаника.

### Цели:
* познакомиться с библиотеками matplotlib, seaborn и визуализациями в pandas
* научиться делать различные визуализации
* заполнять пропуски в данных
* делать однофакторный анализ
* конструировать новые признаки

### Как нужно строить графики:
1. Если график стандартный, используйте matplotlib напрямую из pandas
2. Если график нестандартный, используйте matplotlib
3. Если график совсем нестандартный, то разделите его на несколько стандартных и используйте matplotlib
4. Если нужны профильные красивые графики, график гистограммы с распределением, японские свечи, график pairplot или heatmap, то используйте seaborn (это всё тоже можно сделать в matplotlib, но будет дольше)

### План:
1. Учимся строить графики
2. Обрабатываем признаки
3. Однофакторный анализ

In [ ]:
#Установка seaborn
#!pip install seaborn


In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")


In [ ]:
#print(sns.__version__)

## 1. Учимся строить графики

Сперва загрузим данные и ещё раз изучим базовую информацию при помощи pandas.

In [ ]:
df = pd.read_csv("titanic_train.csv", sep=",")

In [ ]:
print(f"Data size: {df.shape}")
df.head(10)

Каждая строчка набора данных содержит следующие поля:

- Pclass — класс пассажира (1 — высший, 2 — средний, 3 — низший);

- Name — имя;

- Sex — пол;

- Age — возраст;

- SibSp — количество братьев, сестер, сводных братьев, сводных сестер, супругов на борту Титаника;

- Parch — количество родителей, детей (в том числе приемных) на борту Титаника;

- Ticket — номер билета;

- Fare — плата за проезд;

- Cabin — каюта;

- Embarked — порт посадки (C — Шербур; Q — Квинстаун; S — Саутгемптон)

- Survived - пассажир выжил (1) или нет (0).

В поле Age приводится количество полных лет. Для детей меньше 1 года — дробное. Если возраст не известен точно, то указано примерное значение в формате xx.5.

### Описание признаков

In [ ]:
df.describe() #числовые признаки

In [ ]:
df.describe(include = 'object') #категориальные признаки

In [ ]:
#доля пропущенных значений в каждй колонке
df.isna().mean()  # df.isnull() — то же самое, что df.isna()

### Парные отношения в наборе данных  
(диаграммы рассеяния и плотности распределения для всех признаков)

Теперь давайте посмотрим на данные глазами. По диагонали — маргинальное распределение каждой числовой переменной с учётом целевой переменной. В остальных ячейках — scatter plot каждой пары числовых переменных с учётом целевой переменной.

In [ ]:
#sns.pairplot?

In [ ]:
sns.pairplot(df, hue="Survived"); #; - чтобы не смотреть принт, hue - оттенок

### Гистограмма

Начинаем работу с графиками matplotlib и pandas (вызывает те же функции matplotlib).

In [ ]:
#вар 1 (гистограмма по целевой переменной: выжил или нет)
plt.figure(figsize=(15, 5), dpi=300)
plt.title("Survived")
df["Survived"].hist(bins=3); #с помощью pandas (bins - число столбиков)


In [ ]:
#вар 2 (гистограмма по целевой переменной: выжил или нет)
plt.figure(figsize=(8, 3), dpi=300)
plt.title("Survived", fontsize=10)
plt.hist(df["Survived"], bins=3); # То же самое, но только с matplotlib


Построим гистаграмму по возрасту. Обратите внимание, что размер фигуры (и многие другие параметры, например, размер шрифтов) можно задавать один раз на весь нутбук при помощи `plt.rc.Params`.

In [ ]:
#Гистограмма по возрасту
# plt.figure(figsize=(15, 5), dpi=300)
plt.rcParams["figure.figsize"] = (15, 5)
plt.title("Age")
df["Age"].hist(bins=20); #с помощью pandas

Для лёгкого разделения по группам выживших и не выживших, используем `seaborn`.

In [ ]:
#вариант 1 (через seaborn)
sns.displot(df, x="Age", hue="Survived") #по умолчанию - гистограмма (по возрасту, раскраска - по выживаемости)
plt.show()

In [ ]:
#sns.displot?

В `matplotlib` будет дольше и менее красиво:

In [ ]:
#вариант 2 (через matplotlib)
#plt.figure(figsize=(15,5))
plt.title("Age")
plt.hist(
    [df["Age"][df["Survived"] == 1], df["Age"][df["Survived"] == 0]],
    stacked=True,
    bins=20,
);

### Корреляция

Давайте оценим корреляцию между столбцами числовых признаков.

In [ ]:
#Вариант 1 (Матрица корреляции)
df.corr(numeric_only=True) #мера линейной зависимости

Сделаем её чуть более читаемой.

In [ ]:
#Вариант 2 (Матрица корреляции визуально)
sns.heatmap(df.corr(numeric_only=True));

И ещё более читаемой!

In [ ]:
#Вариант 3 (Матрица корреляции визуально) (cmap - цвет, annot - подписи на ячейках, fmt - число знаков после ,)
sns.heatmap(df.corr(numeric_only=True), cmap="vlag", annot=True, fmt="0.2f");

In [ ]:
#Вариант 4 (Матрица корреляции визуально) (отображаем corr, если по модулю больше 0.3)
ax = sns.heatmap(df.corr(numeric_only=True), cmap="vlag", annot=True, fmt="0.2f")
for t in ax.texts:
    if float(t.get_text()) >= 0.3 or float(t.get_text()) <= -0.3:
        t.set_text(t.get_text())
    else:
        t.set_text("")

### Группировки

Построим распредление количества билетов разных классов при помощи **bar plot** (горизонтальный).

In [ ]:
df.groupby("Pclass")["Name"].nunique() #число уникальных значений

In [ ]:
df.groupby("Pclass")["Name"].nunique().sort_values() #сортируем

In [ ]:
#df.plot?

In [ ]:
df.groupby("Pclass")["Name"].nunique().sort_values().plot(kind="barh"); #с помощью pandas (горизонтальный)

### Диаграмма рассеяния (scatter plot)

Посмотрим на взаимосвязь разных переменных при помощи scatter plot (диаграмма рассеяния).

Как scatter строить НЕ нужно:

In [ ]:
plt.scatter(df["Pclass"], df["Survived"]); #Мало смысла (Как scatter строить НЕ нужно)

In [ ]:
plt.scatter(df["Age"], df["Fare"]); #Имеет смысл

### Дополнительные возможности с графиками

Ещё на график можно что-то дорисовать. Например, **дополнительные оси**. Можно задать **цвет**.

In [ ]:
plt.scatter(df["Age"], df["Fare"])#,c=df['Survived']) #(точки можно окрасить в цвет признака c=df['Survived'])
plt.axhline(10, c="y") #оси, цвет = yellow
plt.axvline(10, c="y");


Рисуем **несколько графиков**.

In [ ]:
#Вариант 1
plt.figure(figsize=(15, 5), dpi=300)

plt.subplot(1, 2, 1) #plt.subplot(n_rows, n_cols, index)
plt.title("1")
plt.hist(df["Age"])
plt.ylabel("Count")
plt.xlabel("Age")

plt.subplot(1, 2, 2)
plt.title("2")
plt.hist(df["Fare"])
plt.xlabel("Fare")

plt.show()

Другой способ взаимодействия с подграфиками. Этот способ позволит, например, удобно итерироваться в цикле по графикам и отрисовывать что-то.

In [ ]:
#Вариант 2
fig, axs = plt.subplots(1, 2, figsize=(15, 5), dpi=300)

axs[0].set_title("1")
axs[0].hist(df["Age"])
axs[0].set_ylabel("Count")
axs[0].set_xlabel("Age")

axs[1].set_title("2")
axs[1].hist(df["Fare"])
axs[1].set_xlabel("Fare")

plt.show()

## 2. Обрабатываем признаки

1. Найдем признаки, у которых есть пропущенные значения, и обработаем эти пропущенные значения
2. Переведём категориальные признаки в числовые



**Задание. Ввыведите (описание) основных числовых характеристик числовых столбцов.**

In [ ]:
# <YOUR CODE HERE>

#### <u> Обработаем признак `Sex`</u>


**Задание. Проверьте, есть ли пропуски в столбце `Sex`.**

In [ ]:
# <YOUR CODE HERE>

In [ ]:
print(df['Sex'].unique()) #уникальные значения

Признак `Sex` является категориальным, то есть содержит нечисловые значения. Для работы большинства алгоритмов необходимо переводить категории в числа. Как это можно сделать? 

**Задание. Примените кодировку (для двух значений!), например, мужчина: 1, женщина: 0** (можно применить `map` к столбцу)

In [ ]:
# <YOUR CODE HERE>

После первичной обработки можем посмотреть, как **влияет пол на выживаемость**.

In [ ]:
sns.barplot(x="Sex", y="Survived", data=df, palette="summer") #seaborn
plt.title("Sex - Survived")
plt.show()

Посмотрим, как влияет **пол человека и класс билета (Pclass) на выживаемость**

In [ ]:
sns.barplot(x="Sex", y="Survived", hue="Pclass", data=df, palette="autumn")
plt.title("Sex - Survived")
plt.show()

Ещё один полезный вид визуализации — **ящик с усами**. Такой вид графиков позволяет визуально оценить моду и разброс распределения признака. Посмотрим на ящик с усами, отражающий распределение пассажиров по полу и возрасту.

In [ ]:
sns.catplot(x="Sex", y="Age", data=df, kind="box")  # box plot (box-and-whiskers-plot)
plt.show()

In [ ]:
sns.catplot(x="Sex", y="Age", hue="Pclass", data=df, kind="box")
plt.show()

#### <u>Обработаем признак `Embarked` (порт посадки)</u>

In [ ]:
#Проверяем на пропущенные значения (другой вариант)
print(f"Data size: {df.shape}")
df["Embarked"].value_counts(dropna=False) 
#dropna=False - считает число пропусков !!!

**Задание. Удалите из таблицы пассажиров, для которых неизвестен порт посадки** (используйте `dropna`)

In [ ]:
df.dropna(subset=["Embarked"], inplace=True)
print(f"Data size: {df.shape}")

**Задание. Преобразуем столбец `Embarked` <u>методом OneHot-кодирования</u> при помощи `pd.get_dummies)` (более двух значений!).**

In [ ]:
pd.get_dummies(df, columns = ['Embarked'], dtype = int)

In [ ]:
df = pd.get_dummies(df, columns = ['Embarked'], drop_first=True, dtype = int)
df.head()

In [ ]:
df.shape

#### <u>Обработаем признак `Age`</u>

Проверьте, если ли в `Age` пропущенные значения.

In [ ]:
# <YOUR CODE HERE>

Заполним пропуски медианным (лучше) значением `Age`.

In [ ]:
median_age = df["Age"].median()
df["Age"].fillna(median_age, inplace=True)


Нарисуем распределение возраста пассажиров.

In [ ]:
#сглаженный вариант гистограммы (kde=True)
sns.distplot(df["Age"], kde=True)
plt.show()
#пик, т.к. так заполнили пропуски

---

##### Дополнительно

Посмотрим на распределение `Pclass` по возрастам.

In [ ]:
facet = sns.FacetGrid(data=df, hue="Pclass", legend_out=True, height=5, aspect=1.5)
facet = facet.map(sns.kdeplot, "Age")
facet.add_legend(fontsize=20);

---

#### <u>Обработаем признак `Fare` (тариф)</u>

**Задание. Проверьте, если ли в `Fare` пропущенные значения. Если пропущенные значения есть, заполните их медианными значениями `Fare`**

In [ ]:
# <YOUR CODE HERE>

#### <u>Обработаем признак `Pclass` (класс)</u>

**Задание. Проверьте, если ли в `Pclass` пропущенные значения.**

In [ ]:
# <YOUR CODE HERE>

**Задание. Нарисуйте гистограмму выживаемости в зависимости от `Pclass`** (используйте `sns.barplot`)

In [ ]:
# <YOUR CODE HERE>

#### <u>Обработаем признак `SibSp` (число братьев или сестер, мужей, жен)</u>

Проверим, если ли в `SibSp` пропущенные значения.

In [ ]:
# <YOUR CODE HERE>

#### <u>Обработаем признак `Parch` (количество родителей, детей (в том числе приемных))</u>

Проверим, если ли в `Parch` пропущенные значения.

In [ ]:
# <YOUR CODE HERE>

**Задание. Столбец PassengerId является категориальным и не несёт важной информации, удалите его.**

In [ ]:
# <YOUR CODE HERE>

## 3. Feature engineering

Нарисуем матрицу корреляций числовых признаков между собой и с целевой переменной.

**Задание. Создайте таблицу `NumericData`, которая содержит только числовые столбцы из таблицы `df`** (используйте `df.select_dtypes`)

In [ ]:
# <YOUR CODE HERE>

In [ ]:
colormap = plt.cm.RdBu #цветовая карта
plt.figure(figsize=(14, 12))

plt.title("Pearson Correlation of Features", y=1.05, size=18)
sns.heatmap(
    NumericData.corr(),
    linewidths=0.1,
    vmax=1.0,
    square=True,
    cmap=colormap,
    linecolor="white",
    annot=True,
);

Посмотрим на **попарные зависимости между некоторыми признаками**.

In [ ]:
g = sns.pairplot(
    df[["Survived", "Pclass", "Sex", "Age", "Parch", "Fare"]],
    hue="Survived",
    palette="seismic",
    size=4,
    diag_kind="kde",
    diag_kws=dict(shade=True),
    plot_kws=dict(s=50),
)
g.set(xticklabels=[]);

---

Зачастую признак может быть зашумлен или может содержать лишнюю (слишком детализированную, ведущую к переобучению) информацию. Возможным решением этой проблемы служит бинаризация признака.

**<u>Бинаризуем признаки `Age` и `Fare`.**</u>

* `pd.cut` — разбиение целочисленных данных на несколько интервалов по квантилям
* `pd.qcut` — разбиение числовых (необязательно целочисленных) данных на несколько интервалов по квантилям

In [ ]:
#Разобьем возраст на 5 групп
df["AgeBin"] = pd.cut(df["Age"].astype(int), 5)
df["AgeBin"]#.head()

Теперь переведём полученные интервалы в числа, используя `LabelEncoder`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

#каждой группе поствим в соответствие свое число
label = LabelEncoder()
df["AgeBin_Code"] = label.fit_transform(df["AgeBin"])
df[["Age", "AgeBin", "AgeBin_Code"]].head()

In [ ]:
import numpy as np

In [ ]:
#коэффициенты корреляции
np.corrcoef(df['Age'],df['Survived'])[0][1]

In [ ]:
#коэффициенты корреляции
np.corrcoef(df['AgeBin_Code'],df['Survived'])[0][1]

**Задание. Бинаризуйте `Fare`, используя разбиение на 4 интервала.**

In [ ]:
# <YOUR CODE HERE>

---

## Дополнительно

Мы не используем всю информацию о данных, в частности, не используем текстовые данные. Также из матрицы корреляций мы видим, что признаки `Parch` и `SibSp` слабо коррелируют с выживаемостью (`Survived`). Сконструируем новые признаки, чтобы решить эти проблемы.

**Задание.**

1) Создайте признак `NameLen` и запишите в него длину имени (Name) (можно использовать `lambda`, `apply`) 

2) Создайте признак `FamilySize`, равный `Parch` + `SibSp` + 1(сам человек)

3) Создайте признак `IsAlone`, который показывает, путешествовал человек один или с семьей (можно использовать `lambda`, `apply`)

In [ ]:
# <YOUR CODE HERE>

**Задание.** Посмотрите, как коррелируют новые признаки (не забудьте включить сюда бинаризованные признаки) со столбцом `Survived`.

In [ ]:
# <YOUR CODE HERE>

Можно извлечь и другую полезную информацию из данных путём конструирования новых признаков. 

**Задание.** Придумайте новые осмысленные признаки. Проверьте, как они коррелируют с выживаемостью.

In [ ]:
# <YOUR CODE HERE>

**Задание.** Верно ли, что если признак имеет маленькую по модулю корреляцию с выживаемостью, то он не влияет на неё и бесполезен для нашей задачи?

Мы провели довольно подробный однофакторный анализ данных и увидели, какие признаки сильно влияют на выживаемость, а какие нет. 

**Задание.**

a) Попробуйте **написать свою модель** для предсказания выживаемости, используя обнаруженные закономерности (напрмер, по полу `Sex` выживаемость `Survived`)

b) **Оцените качество модели**: вычислите долю правильных ответов алгоритма по всем данным

In [ ]:
# <YOUR CODE HERE>